In [36]:
import urllib
import requests
from bs4 import BeautifulSoup
import time
import tqdm
import pandas as pd
import itertools
import urllib.request
import numpy as np
import json
import pickle


from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options  
from selenium.webdriver.support.ui import Select

import html
import os

#### First source: http://orientalbirdimages.org/birdimages.php

In [19]:
def form_group_url(group_id):
    return "http://orientalbirdimages.org/birdimages.php?action=birdgroup&Bird_Group_ID=%d"%group_id

def form_family_url(group_id):
    return "http://orientalbirdimages.org/birdimages.php?action=birdfamily&Bird_Family_ID=%d"%group_id

def form_image_url(bird_id, image_id):
    return "http://orientalbirdimages.org/search.php?Bird_ID=%d&Bird_Image_ID=%d"%(bird_id, image_id)

In [3]:
def pickle_obj(obj, fname):
    with open(fname, 'wb') as f:
        pickle.dump(obj, f)
    
def try_unpickle(fname):
    try:
        return unpickle_obj(fname)
    except FileNotFoundError:
        return []
    
def unpickle_obj(fname):
    with open(fname, 'rb') as f:
        return pickle.load(f)

In [4]:
valid_groups = try_unpickle('valid_groups.pkl')
valid_families = try_unpickle('valid_families.pkl')
valid_ids = try_unpickle('valid_ids.pkl')

In [5]:
# chrome_options = Options()  
# chrome_options.add_argument("--headless")
driver = Chrome(executable_path=r'../chromedriver_win32/chromedriver.exe')#, options = chrome_options)

In [6]:
base_url = 'http://orientalbirdimages.org/birdimages.php'

In [7]:
if len(valid_groups)==0:

    num_elements = int(len(driver.find_elements_by_class_name("detail")) / 2)

    for i in tqdm.tqdm_notebook(range(num_elements)):
        driver.get(base_url)
        time.sleep(2)
        driver.find_elements_by_class_name("detail")[2*i].click()
        curr_url = driver.current_url
        time.sleep(2)
        this_group_id = int(curr_url[curr_url.rfind('Bird_Group_ID') + len('Bird_Group_ID='):])
        valid_groups.append(this_group_id)
    

In [8]:
pickle_obj(valid_groups, 'valid_groups.pkl')

In [9]:
if len(valid_families)==0:
    pbar = tqdm.tqdm_notebook(total = len(valid_groups))
    for group in valid_groups:
        this_url = form_group_url(group)
        driver.get(this_url)
        num_elements = int(len(driver.find_elements_by_class_name("detail")) / 2)
        
        for i in tqdm.tqdm_notebook(range(num_elements), leave=False):
            driver.get(this_url)
            time.sleep(2)
            driver.find_elements_by_class_name("detail")[2*i].click()
            time.sleep(2)
            curr_url = driver.current_url
            if curr_url.find('Bird_Group_ID') > 0:
                continue
            this_family = int(curr_url[curr_url.rfind('Bird_Family_ID') + len('Bird_Family_ID='):])
            valid_families.append(this_family)
            
        pbar.update(1)
            
            

In [10]:
pickle_obj(valid_families, 'valid_families.pkl')

In [11]:
test_url = form_family_url(valid_families[0])
driver.get(test_url)

In [12]:
len(driver.find_elements_by_class_name("mlink"))

102

In [13]:
if len(valid_ids)==0:
    pbar = tqdm.notebook.tqdm(total = len(valid_families))
    for family in valid_families:
        this_url = form_family_url(family)
        driver.get(this_url)
        num_elements = int(len(driver.find_elements_by_class_name("detail"))/2)
        
        for i in tqdm.notebook.tqdm(range(num_elements), leave=False):
            driver.get(this_url)
            time.sleep(2)
            driver.find_elements_by_class_name("mlink")[2*i].click()
            time.sleep(2)
            curr_url = driver.current_url
            if curr_url.find("Bird_Family_ID") > 0:
                continue
            
            curr_url = curr_url[curr_url.find("?")+1:]
            bird_id, image_id = curr_url.split('&')
            bird_id = int(bird_id[bird_id.find('Bird_ID=') + len('Bird_ID='):])
            image_id = int(image_id[image_id.find('Bird_Image_ID=')+len('Bird_Image_ID='):])
            valid_ids.append((bird_id, image_id))
        pbar.update(1)

C:\Users\Sam\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


C:\Users\Sam\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [15]:
pickle_obj(valid_ids, 'valid_ids.pkl')

In [70]:
len(valid_ids)

2944

In [71]:
valid_ids[0]

(208, 7105)

In [74]:
image_id_dict = {}

pbar = tqdm.notebook.tqdm(total = len(valid_ids))
for bird_id, image_id in valid_ids:
    this_url = form_image_url(bird_id, image_id)
    this_bird_image_ids = []
    driver.get(this_url)
    
    image_list = Select(driver.find_elements_by_class_name("text250")[0])
    for option in image_list.options:
        this_bird_image_ids.append(option.get_property('value'))
    
    def process_id(num):
        if num.find('&') > 0:
            return int(num[:num.find('&')])
        else:
            return int(num)
    
    this_bird_image_ids = [process_id(num) for num in this_bird_image_ids]
    
    image_id_dict[bird_id] = this_bird_image_ids
    
    time.sleep(1)
    pbar.update(1)

C:\Users\Sam\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [76]:
with open('image_dict.json', 'w') as f:
    json.dump(image_id_dict, f)

In [78]:
df_rows = []
for bird_id, image_id_list in image_id_dict.items():
    for image_id in image_id_list:
        df_rows.append([bird_id, image_id, form_image_url(bird_id, image_id)])
        

In [79]:
len(df_rows)

171711

In [81]:
df = pd.DataFrame.from_records(df_rows, columns =  ['bird_id', 'image_id', 'url'])

In [82]:
df.shape

(171711, 3)

In [85]:
df.to_csv('../../data/oriental_bird_database/labels.csv')